# Model Output Walkthrough: History-Centric Next-Location Prediction

This notebook provides a complete, step-by-step walkthrough of how the History-Centric Next-Location Prediction model processes input data and generates predictions. The notebook is **self-contained** and does not depend on any external project scripts - all necessary code is included directly.

## Purpose and Overview

This notebook demonstrates:
1. **Data Loading**: How trajectory sequences are loaded and prepared
2. **Model Architecture**: The structure and components of the History-Centric model
3. **Input Processing**: How raw trajectory data is transformed into model inputs
4. **Forward Pass**: Step-by-step execution through the model layers
5. **History Scoring**: How the model leverages visit history for predictions
6. **Output Generation**: How final location predictions are computed
7. **Evaluation**: How we measure model performance using various metrics

## Key Insight

The model is based on a critical observation: **83.81% of next locations are already in the visit history**. Therefore, the model combines history-based scoring with learned patterns to make accurate predictions.

---

## Step 1: Import Required Libraries

We'll need PyTorch for the neural network, NumPy for numerical operations, and scikit-learn for evaluation metrics.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pickle
import math
from sklearn.metrics import f1_score
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Step 2: Define the History-Centric Model Architecture

The **HistoryCentricModel** combines two strategies:

### A. History-Based Scoring
- **Recency**: More recent visits are weighted higher (exponential decay)
- **Frequency**: Locations visited more often get higher scores
- **Scale Factor**: Amplifies history scores to dominate predictions

### B. Learned Transformer Component
- **Embeddings**: Encode locations, users, and temporal features
- **Self-Attention**: Captures sequential patterns in visit history
- **Feedforward Network**: Learns complex relationships

### Ensemble Strategy
The final prediction combines:
- History scores (weighted heavily - ~11x)
- Learned model scores (weighted lightly - ~0.22x)

This architecture is designed to stay under 500K parameters while maximizing accuracy.

In [ ]:
class HistoryCentricModel(nn.Module):
    """
    Model that heavily prioritizes locations from visit history.
    Combines history-based scoring with learned transformer patterns.
    """
    
    def __init__(self, num_locations, num_users):
        super().__init__()
        
        self.num_locations = num_locations
        self.d_model = 80  # Compact dimensionality
        
        # === EMBEDDING LAYERS ===
        # Location embedding: maps each location ID to a 56-dim vector
        self.loc_emb = nn.Embedding(num_locations, 56, padding_idx=0)
        
        # User embedding: maps each user ID to a 12-dim vector
        self.user_emb = nn.Embedding(num_users, 12, padding_idx=0)
        
        # Temporal feature projection: 6 temporal features -> 12-dim
        # Input features: sin(time), cos(time), duration, sin(weekday), cos(weekday), time_gap
        self.temporal_proj = nn.Linear(6, 12)
        
        # Input normalization: 56 + 12 + 12 = 80
        self.input_norm = nn.LayerNorm(80)
        
        # === POSITIONAL ENCODING ===
        # Sinusoidal positional encoding (max sequence length = 60)
        pe = torch.zeros(60, 80)
        position = torch.arange(0, 60, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, 80, 2).float() * (-math.log(10000.0) / 80))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
        
        # === TRANSFORMER LAYER ===
        # Single-layer multi-head attention (4 heads, 80-dim)
        self.attn = nn.MultiheadAttention(80, 4, dropout=0.35, batch_first=True)
        
        # Feedforward network: 80 -> 160 -> 80
        self.ff = nn.Sequential(
            nn.Linear(80, 160),
            nn.GELU(),
            nn.Dropout(0.35),
            nn.Linear(160, 80)
        )
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(80)
        self.norm2 = nn.LayerNorm(80)
        self.dropout = nn.Dropout(0.35)
        
        # === PREDICTION HEAD ===
        # Maps final hidden state to location scores
        self.predictor = nn.Sequential(
            nn.Linear(80, 160),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(160, num_locations)
        )
        
        # === HISTORY SCORING PARAMETERS (Learnable) ===
        # These are learned during training to optimize the ensemble
        self.recency_decay = nn.Parameter(torch.tensor(0.62))    # How fast recent visits decay
        self.freq_weight = nn.Parameter(torch.tensor(2.2))       # Weight for visit frequency
        self.history_scale = nn.Parameter(torch.tensor(11.0))    # Overall history importance
        self.model_weight = nn.Parameter(torch.tensor(0.22))     # Weight for learned model
        
        self._init_weights()
    
    def _init_weights(self):
        """Initialize model weights using Xavier initialization."""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Embedding):
                nn.init.normal_(m.weight, mean=0, std=0.01)
                if m.padding_idx is not None:
                    m.weight.data[m.padding_idx].zero_()
    
    def compute_history_scores(self, loc_seq, mask):
        """
        Compute history-based scores for all locations.
        
        Strategy:
        1. For each location in the history, compute recency score (exponential decay)
        2. Compute frequency score (how often visited)
        3. Combine: history_score = recency + freq_weight * frequency
        4. Scale by history_scale to amplify importance
        
        Args:
            loc_seq: (batch_size, seq_len) - sequence of location IDs
            mask: (batch_size, seq_len) - True for valid positions
        
        Returns:
            history_scores: (batch_size, num_locations) - score for each location
        """
        batch_size, seq_len = loc_seq.shape
        
        # Initialize score matrices
        recency_scores = torch.zeros(batch_size, self.num_locations, device=loc_seq.device)
        frequency_scores = torch.zeros(batch_size, self.num_locations, device=loc_seq.device)
        
        # Process each timestep in the sequence
        for t in range(seq_len):
            locs_t = loc_seq[:, t]  # (batch_size,) - locations at time t
            valid_t = mask[:, t].float()  # (batch_size,) - mask for valid entries
            
            # Recency: exponential decay from the end of sequence
            # More recent visits have higher scores
            time_from_end = seq_len - t - 1
            recency_weight = torch.pow(self.recency_decay, time_from_end)
            
            # Update recency scores (keep maximum for each location)
            # This ensures the most recent visit dominates
            indices = locs_t.unsqueeze(1)  # (batch_size, 1)
            values = (recency_weight * valid_t).unsqueeze(1)  # (batch_size, 1)
            
            current_scores = torch.zeros(batch_size, self.num_locations, device=loc_seq.device)
            current_scores.scatter_(1, indices, values)
            recency_scores = torch.maximum(recency_scores, current_scores)
            
            # Update frequency scores (sum over all timesteps)
            # This counts how many times each location appears
            frequency_scores.scatter_add_(1, indices, valid_t.unsqueeze(1))
        
        # Normalize frequency scores to [0, 1] range
        max_freq = frequency_scores.max(dim=1, keepdim=True)[0].clamp(min=1.0)
        frequency_scores = frequency_scores / max_freq
        
        # Combine recency and frequency, then scale
        history_scores = recency_scores + self.freq_weight * frequency_scores
        history_scores = self.history_scale * history_scores
        
        return history_scores
    
    def forward(self, loc_seq, user_seq, weekday_seq, start_min_seq, dur_seq, diff_seq, mask):
        """
        Forward pass through the model.
        
        Args:
            loc_seq: (batch_size, seq_len) - location IDs
            user_seq: (batch_size, seq_len) - user IDs
            weekday_seq: (batch_size, seq_len) - weekday (0-6)
            start_min_seq: (batch_size, seq_len) - start time in minutes from midnight
            dur_seq: (batch_size, seq_len) - duration at each location
            diff_seq: (batch_size, seq_len) - time gap indicator
            mask: (batch_size, seq_len) - attention mask
        
        Returns:
            final_logits: (batch_size, num_locations) - scores for each location
        """
        batch_size, seq_len = loc_seq.shape
        
        # === STEP 1: Compute History-Based Scores ===
        history_scores = self.compute_history_scores(loc_seq, mask)
        
        # === STEP 2: Learned Model Path ===
        
        # 2a. Embed locations and users
        loc_emb = self.loc_emb(loc_seq)  # (batch_size, seq_len, 56)
        user_emb = self.user_emb(user_seq)  # (batch_size, seq_len, 12)
        
        # 2b. Encode temporal features using circular encoding
        # Time of day: convert minutes to hours, then to radians
        hours = start_min_seq / 60.0
        time_rad = (hours / 24.0) * 2 * math.pi
        time_sin = torch.sin(time_rad)
        time_cos = torch.cos(time_rad)
        
        # Duration: log-transform and normalize
        dur_norm = torch.log1p(dur_seq) / 8.0
        
        # Weekday: circular encoding
        wd_rad = (weekday_seq.float() / 7.0) * 2 * math.pi
        wd_sin = torch.sin(wd_rad)
        wd_cos = torch.cos(wd_rad)
        
        # Time gap: normalize
        diff_norm = diff_seq.float() / 7.0
        
        # Stack all temporal features: (batch_size, seq_len, 6)
        temporal_feats = torch.stack([time_sin, time_cos, dur_norm, wd_sin, wd_cos, diff_norm], dim=-1)
        temporal_emb = self.temporal_proj(temporal_feats)  # (batch_size, seq_len, 12)
        
        # 2c. Combine all features
        x = torch.cat([loc_emb, user_emb, temporal_emb], dim=-1)  # (batch_size, seq_len, 80)
        x = self.input_norm(x)
        
        # 2d. Add positional encoding
        x = x + self.pe[:seq_len, :].unsqueeze(0)  # Add position information
        x = self.dropout(x)
        
        # 2e. Transformer layer with self-attention
        attn_mask = ~mask  # Invert mask (True = attend, False = ignore)
        attn_out, _ = self.attn(x, x, x, key_padding_mask=attn_mask)
        x = self.norm1(x + self.dropout(attn_out))  # Residual connection
        
        # 2f. Feedforward network
        ff_out = self.ff(x)
        x = self.norm2(x + self.dropout(ff_out))  # Residual connection
        
        # 2g. Extract last valid position for each sequence
        seq_lens = mask.sum(dim=1) - 1  # Get index of last valid position
        indices_gather = seq_lens.unsqueeze(1).unsqueeze(2).expand(batch_size, 1, self.d_model)
        last_hidden = torch.gather(x, 1, indices_gather).squeeze(1)  # (batch_size, d_model)
        
        # 2h. Generate learned logits
        learned_logits = self.predictor(last_hidden)  # (batch_size, num_locations)
        
        # === STEP 3: Ensemble History + Learned ===
        # Normalize learned logits to similar scale as history scores
        learned_logits_normalized = F.softmax(learned_logits, dim=1) * self.num_locations
        
        # Combine with learned weights
        final_logits = history_scores + self.model_weight * learned_logits_normalized
        
        return final_logits
    
    def count_parameters(self):
        """Count trainable parameters."""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Print model summary
print("Model class defined successfully!")

## Step 3: Define Evaluation Metrics

We use several metrics to evaluate next-location prediction:

1. **Accuracy@K**: Is the correct location in the top-K predictions?
   - Acc@1: Exact match (most important)
   - Acc@5, Acc@10: More lenient measures

2. **MRR (Mean Reciprocal Rank)**: Rewards predictions where the correct location appears earlier
   - Score = 1/rank (1.0 for rank 1, 0.5 for rank 2, 0.33 for rank 3, etc.)

3. **NDCG (Normalized Discounted Cumulative Gain)**: Similar to MRR but uses logarithmic discount
   - More commonly used in information retrieval

4. **F1 Score**: Harmonic mean of precision and recall (for top-1 predictions)

In [ ]:
def get_mrr(prediction, targets):
    """
    Calculate Mean Reciprocal Rank.
    
    Args:
        prediction: (batch_size, num_locations) - model output scores
        targets: (batch_size,) - true location IDs
    
    Returns:
        Sum of reciprocal ranks (divide by batch_size for mean)
    """
    # Sort predictions in descending order to get rankings
    index = torch.argsort(prediction, dim=-1, descending=True)
    
    # Find where the target appears in the ranking
    hits = (targets.unsqueeze(-1).expand_as(index) == index).nonzero()
    
    # Get ranks (1-indexed)
    ranks = (hits[:, -1] + 1).float()
    
    # Compute reciprocal ranks
    rranks = torch.reciprocal(ranks)
    
    return torch.sum(rranks).cpu().numpy()


def get_ndcg(prediction, targets, k=10):
    """
    Calculate Normalized Discounted Cumulative Gain.
    
    Args:
        prediction: (batch_size, num_locations) - model output scores
        targets: (batch_size,) - true location IDs
        k: Consider only top-k predictions
    
    Returns:
        Sum of NDCG scores
    """
    # Sort predictions in descending order
    index = torch.argsort(prediction, dim=-1, descending=True)
    
    # Find where the target appears in the ranking
    hits = (targets.unsqueeze(-1).expand_as(index) == index).nonzero()
    ranks = (hits[:, -1] + 1).float().cpu().numpy()
    
    # Compute DCG with logarithmic discount
    # Only consider ranks <= k
    not_considered_idx = ranks > k
    ndcg = 1 / np.log2(ranks + 1)
    ndcg[not_considered_idx] = 0
    
    return np.sum(ndcg)


def calculate_correct_total_prediction(logits, true_y):
    """
    Calculate comprehensive prediction metrics.
    
    Args:
        logits: (batch_size, num_locations) - model output
        true_y: (batch_size,) - ground truth location IDs
    
    Returns:
        result_array: [correct@1, correct@3, correct@5, correct@10, rr, ndcg, total]
        true_y_cpu: Ground truth on CPU
        top1: Top-1 predictions
    """
    top1 = []
    result_ls = []
    
    # Calculate Acc@K for K in [1, 3, 5, 10]
    for k in [1, 3, 5, 10]:
        # Handle case where num_locations < k
        if logits.shape[-1] < k:
            k = logits.shape[-1]
        
        # Get top-k predictions
        prediction = torch.topk(logits, k=k, dim=-1).indices
        
        # Save top-1 for F1 calculation
        if k == 1:
            top1 = torch.squeeze(prediction).cpu()
        
        # Count how many times true label is in top-k
        top_k = torch.eq(true_y[:, None], prediction).any(dim=1).sum().cpu().numpy()
        result_ls.append(top_k)
    
    # Add MRR
    result_ls.append(get_mrr(logits, true_y))
    
    # Add NDCG
    result_ls.append(get_ndcg(logits, true_y))
    
    # Add total count
    result_ls.append(true_y.shape[0])
    
    return np.array(result_ls, dtype=np.float32), true_y.cpu(), top1


def get_performance_dict(return_dict):
    """
    Convert raw metric counts to percentages.
    
    Args:
        return_dict: Dictionary with counts
    
    Returns:
        Dictionary with accuracy percentages
    """
    perf = {
        "correct@1": return_dict["correct@1"],
        "correct@3": return_dict["correct@3"],
        "correct@5": return_dict["correct@5"],
        "correct@10": return_dict["correct@10"],
        "rr": return_dict["rr"],
        "ndcg": return_dict["ndcg"],
        "f1": return_dict.get("f1", 0),
        "total": return_dict["total"],
    }
    
    # Convert to percentages
    perf["acc@1"] = perf["correct@1"] / perf["total"] * 100
    perf["acc@5"] = perf["correct@5"] / perf["total"] * 100
    perf["acc@10"] = perf["correct@10"] / perf["total"] * 100
    perf["mrr"] = perf["rr"] / perf["total"] * 100
    perf["ndcg"] = perf["ndcg"] / perf["total"] * 100
    
    return perf

print("Metric functions defined successfully!")

## Step 4: Load Test Data

The data is stored in pickle format with preprocessed trajectory sequences. Each sample contains:

- **X**: Sequence of location IDs (variable length)
- **user_X**: User ID for each visit
- **weekday_X**: Day of week (0=Monday, 6=Sunday)
- **start_min_X**: Start time in minutes from midnight (0-1439)
- **dur_X**: Duration at each location in minutes
- **diff**: Time gap indicator between consecutive visits
- **Y**: Next location (target to predict)

The sequences are padded to the same length within each batch for efficient processing.

In [ ]:
# Load test data
data_path = '/content/expr_hrcl_next_pred_av5/data/geolife/geolife_transformer_7_test.pk'

with open(data_path, 'rb') as f:
    test_data = pickle.load(f)

print(f"Loaded {len(test_data)} test samples")
print(f"\nExample sample structure:")
sample = test_data[0]
for key, value in sample.items():
    if isinstance(value, (list, np.ndarray)):
        print(f"  {key}: length={len(value)}, sample={value[:3] if len(value) > 3 else value}")
    else:
        print(f"  {key}: {value}")

## Step 5: Define Dataset Class

The dataset class handles:
1. Loading samples from the pickle file
2. Truncating long sequences to max length (keeping most recent visits)
3. Converting to PyTorch tensors with appropriate data types

In [ ]:
from torch.utils.data import Dataset, DataLoader

class GeoLifeDataset(Dataset):
    """Dataset for GeoLife trajectory sequences."""
    
    def __init__(self, data, max_seq_len=60):
        self.data = data
        self.max_seq_len = max_seq_len
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        
        # Extract features
        loc_seq = sample['X']
        user_seq = sample['user_X']
        weekday_seq = sample['weekday_X']
        start_min_seq = sample['start_min_X']
        dur_seq = sample['dur_X']
        diff_seq = sample['diff']
        target = sample['Y']
        
        # Truncate if too long (keep most recent)
        seq_len = len(loc_seq)
        if seq_len > self.max_seq_len:
            loc_seq = loc_seq[-self.max_seq_len:]
            user_seq = user_seq[-self.max_seq_len:]
            weekday_seq = weekday_seq[-self.max_seq_len:]
            start_min_seq = start_min_seq[-self.max_seq_len:]
            dur_seq = dur_seq[-self.max_seq_len:]
            diff_seq = diff_seq[-self.max_seq_len:]
            seq_len = self.max_seq_len
        
        return {
            'loc_seq': torch.LongTensor(loc_seq),
            'user_seq': torch.LongTensor(user_seq),
            'weekday_seq': torch.LongTensor(weekday_seq),
            'start_min_seq': torch.FloatTensor(start_min_seq),
            'dur_seq': torch.FloatTensor(dur_seq),
            'diff_seq': torch.LongTensor(diff_seq),
            'target': torch.LongTensor([target]),
            'seq_len': seq_len
        }


def collate_fn(batch):
    """
    Collate function to pad variable-length sequences.
    Creates a batch with uniform sequence length.
    """
    # Find max length in this batch
    max_len = max(item['seq_len'] for item in batch)
    batch_size = len(batch)
    
    # Initialize padded tensors (all zeros initially)
    loc_seqs = torch.zeros(batch_size, max_len, dtype=torch.long)
    user_seqs = torch.zeros(batch_size, max_len, dtype=torch.long)
    weekday_seqs = torch.zeros(batch_size, max_len, dtype=torch.long)
    start_min_seqs = torch.zeros(batch_size, max_len, dtype=torch.float)
    dur_seqs = torch.zeros(batch_size, max_len, dtype=torch.float)
    diff_seqs = torch.zeros(batch_size, max_len, dtype=torch.long)
    targets = torch.zeros(batch_size, dtype=torch.long)
    seq_lens = torch.zeros(batch_size, dtype=torch.long)
    
    # Fill in the actual data
    for i, item in enumerate(batch):
        length = item['seq_len']
        loc_seqs[i, :length] = item['loc_seq']
        user_seqs[i, :length] = item['user_seq']
        weekday_seqs[i, :length] = item['weekday_seq']
        start_min_seqs[i, :length] = item['start_min_seq']
        dur_seqs[i, :length] = item['dur_seq']
        diff_seqs[i, :length] = item['diff_seq']
        targets[i] = item['target']
        seq_lens[i] = length
    
    # Create attention mask (True for real tokens, False for padding)
    mask = torch.arange(max_len).unsqueeze(0) < seq_lens.unsqueeze(1)
    
    return {
        'loc_seq': loc_seqs,
        'user_seq': user_seqs,
        'weekday_seq': weekday_seqs,
        'start_min_seq': start_min_seqs,
        'dur_seq': dur_seqs,
        'diff_seq': diff_seqs,
        'target': targets,
        'mask': mask,
        'seq_len': seq_lens
    }

# Create dataset and dataloader
test_dataset = GeoLifeDataset(test_data, max_seq_len=60)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0
)

print(f"Created test dataset with {len(test_dataset)} samples")
print(f"Number of batches: {len(test_loader)}")

## Step 6: Load Trained Model

We load the pre-trained model weights from the checkpoint. The model was trained with:
- **962 locations** (unique places in GeoLife dataset)
- **48 users** (study participants)
- Early stopping based on validation loss
- Label smoothing for better generalization

In [ ]:
# Model configuration
num_locations = 962
num_users = 48

# Initialize model
model = HistoryCentricModel(num_locations, num_users)
model = model.to(device)

# Load trained weights
# Note: If checkpoint loading fails due to dependencies, the model can still run
# with random weights to demonstrate the architecture
checkpoint_path = '/content/expr_hrcl_next_pred_av5/trained_models/best_model.pt'

try:
    # Try to load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    
    # Load only the model state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()  # Set to evaluation mode
    
    print(f"Model loaded successfully!")
    print(f"\nModel Statistics:")
    print(f"  Total parameters: {model.count_parameters():,}")
    print(f"  Trained epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"  Validation accuracy: {checkpoint.get('val_acc', 'N/A'):.2f}%")
    print(f"\nLearned Ensemble Weights:")
    print(f"  Recency decay: {model.recency_decay.item():.4f}")
    print(f"  Frequency weight: {model.freq_weight.item():.4f}")
    print(f"  History scale: {model.history_scale.item():.4f}")
    print(f"  Model weight: {model.model_weight.item():.4f}")
except Exception as e:
    print(f"Could not load checkpoint: {e}")
    print("\nUsing initialized model with default parameters.")
    print("To see full trained performance, train the model using train_model.py")
    print(f"\nModel Statistics:")
    print(f"  Total parameters: {model.count_parameters():,}")
    model.eval()

## Step 7: Detailed Walkthrough - Single Sample

Let's process a single sample step-by-step to understand exactly what the model does.

### Input Breakdown
We'll examine each component of the input and trace it through the model.

In [ ]:
# Get a single batch for detailed analysis
sample_batch = next(iter(test_loader))

# Extract first sample from batch
idx = 0
print("=" * 80)
print("SAMPLE INPUT FEATURES")
print("=" * 80)

# Get sequence length for this sample
seq_len = sample_batch['seq_len'][idx].item()
print(f"\nSequence Length: {seq_len}")

# Location sequence
loc_seq = sample_batch['loc_seq'][idx, :seq_len]
print(f"\nLocation Sequence (last 10): {loc_seq[-10:].tolist()}")

# User sequence
user_seq = sample_batch['user_seq'][idx, :seq_len]
print(f"User ID: {user_seq[0].item()} (constant for this sequence)")

# Temporal features
weekday_seq = sample_batch['weekday_seq'][idx, :seq_len]
print(f"\nWeekdays (last 10): {weekday_seq[-10:].tolist()}")

start_min_seq = sample_batch['start_min_seq'][idx, :seq_len]
print(f"Start times in minutes (last 10): {start_min_seq[-10:].tolist()}")

dur_seq = sample_batch['dur_seq'][idx, :seq_len]
print(f"Durations (last 10): {dur_seq[-10:].tolist()}")

diff_seq = sample_batch['diff_seq'][idx, :seq_len]
print(f"Time gaps (last 10): {diff_seq[-10:].tolist()}")

# Target
target = sample_batch['target'][idx]
print(f"\nTarget (next location to predict): {target.item()}")

# Check if target is in history
is_in_history = target.item() in loc_seq.tolist()
print(f"Is target in visit history? {is_in_history}")

## Step 8: History-Based Scoring Mechanism

This is the core innovation of the model. We compute scores for all locations based on:
1. **When** they were last visited (recency)
2. **How often** they were visited (frequency)

Let's trace through this process step-by-step.

In [ ]:
# Move sample to device
loc_seq_device = sample_batch['loc_seq'][[idx]].to(device)
mask_device = sample_batch['mask'][[idx]].to(device)

# Compute history scores
with torch.no_grad():
    history_scores = model.compute_history_scores(loc_seq_device, mask_device)

history_scores_np = history_scores[0].cpu().numpy()

print("=" * 80)
print("HISTORY SCORING ANALYSIS")
print("=" * 80)

# Find non-zero scores (locations in history)
nonzero_indices = np.where(history_scores_np > 0)[0]
print(f"\nNumber of locations in history: {len(nonzero_indices)}")
print(f"Total locations in dataset: {num_locations}")

# Get top scored locations from history
top_k = 10
top_indices = np.argsort(history_scores_np)[-top_k:][::-1]
top_scores = history_scores_np[top_indices]

print(f"\nTop {top_k} locations by history score:")
for i, (loc_id, score) in enumerate(zip(top_indices, top_scores), 1):
    in_seq = "✓" if loc_id in loc_seq.tolist() else "✗"
    target_marker = "← TARGET" if loc_id == target.item() else ""
    print(f"  {i}. Location {loc_id:3d}: score={score:6.2f} {in_seq} {target_marker}")

# Check target location score
target_score = history_scores_np[target.item()]
target_rank = np.sum(history_scores_np > target_score) + 1
print(f"\nTarget location score: {target_score:.2f}")
print(f"Target rank (by history alone): {target_rank}")

## Step 9: Learned Model Processing

Now let's see what the transformer-based learned component does:

1. **Embedding**: Convert IDs to dense vectors
2. **Temporal Encoding**: Encode time features using circular representations
3. **Self-Attention**: Capture sequential patterns
4. **Prediction**: Generate scores for all locations

In [ ]:
# Prepare full batch inputs
loc_seq_batch = sample_batch['loc_seq'].to(device)
user_seq_batch = sample_batch['user_seq'].to(device)
weekday_seq_batch = sample_batch['weekday_seq'].to(device)
start_min_seq_batch = sample_batch['start_min_seq'].to(device)
dur_seq_batch = sample_batch['dur_seq'].to(device)
diff_seq_batch = sample_batch['diff_seq'].to(device)
mask_batch = sample_batch['mask'].to(device)

print("=" * 80)
print("LEARNED MODEL PROCESSING")
print("=" * 80)

# Forward pass with intermediate outputs
with torch.no_grad():
    batch_size, seq_len_max = loc_seq_batch.shape
    
    # Step 1: Embeddings
    loc_emb = model.loc_emb(loc_seq_batch)
    user_emb = model.user_emb(user_seq_batch)
    print(f"\n1. Embedding Dimensions:")
    print(f"   Location embedding: {loc_emb.shape}")
    print(f"   User embedding: {user_emb.shape}")
    
    # Step 2: Temporal features
    hours = start_min_seq_batch / 60.0
    time_rad = (hours / 24.0) * 2 * math.pi
    time_sin = torch.sin(time_rad)
    time_cos = torch.cos(time_rad)
    dur_norm = torch.log1p(dur_seq_batch) / 8.0
    wd_rad = (weekday_seq_batch.float() / 7.0) * 2 * math.pi
    wd_sin = torch.sin(wd_rad)
    wd_cos = torch.cos(wd_rad)
    diff_norm = diff_seq_batch.float() / 7.0
    
    temporal_feats = torch.stack([time_sin, time_cos, dur_norm, wd_sin, wd_cos, diff_norm], dim=-1)
    temporal_emb = model.temporal_proj(temporal_feats)
    print(f"   Temporal embedding: {temporal_emb.shape}")
    
    # Step 3: Combine features
    x = torch.cat([loc_emb, user_emb, temporal_emb], dim=-1)
    x = model.input_norm(x)
    print(f"\n2. Combined features: {x.shape}")
    
    # Step 4: Add positional encoding
    x = x + model.pe[:seq_len_max, :].unsqueeze(0)
    x = model.dropout(x)
    print(f"   After positional encoding: {x.shape}")
    
    # Step 5: Self-attention
    attn_mask = ~mask_batch
    attn_out, attn_weights = model.attn(x, x, x, key_padding_mask=attn_mask)
    x = model.norm1(x + model.dropout(attn_out))
    print(f"\n3. After self-attention: {x.shape}")
    
    # Step 6: Feedforward
    ff_out = model.ff(x)
    x = model.norm2(x + model.dropout(ff_out))
    print(f"   After feedforward: {x.shape}")
    
    # Step 7: Extract last hidden state
    seq_lens = mask_batch.sum(dim=1) - 1
    indices_gather = seq_lens.unsqueeze(1).unsqueeze(2).expand(batch_size, 1, model.d_model)
    last_hidden = torch.gather(x, 1, indices_gather).squeeze(1)
    print(f"\n4. Last hidden state: {last_hidden.shape}")
    
    # Step 8: Prediction head
    learned_logits = model.predictor(last_hidden)
    print(f"   Learned logits: {learned_logits.shape}")
    
    # Get learned predictions for first sample
    learned_logits_sample = learned_logits[0].cpu().numpy()
    
# Analyze learned model predictions
top_learned_indices = np.argsort(learned_logits_sample)[-10:][::-1]
top_learned_scores = learned_logits_sample[top_learned_indices]

print(f"\nTop 10 locations by learned model:")
for i, (loc_id, score) in enumerate(zip(top_learned_indices, top_learned_scores), 1):
    in_seq = "✓" if loc_id in loc_seq.tolist() else "✗"
    target_marker = "← TARGET" if loc_id == target.item() else ""
    print(f"  {i}. Location {loc_id:3d}: score={score:7.2f} {in_seq} {target_marker}")

learned_target_score = learned_logits_sample[target.item()]
learned_target_rank = np.sum(learned_logits_sample > learned_target_score) + 1
print(f"\nTarget rank (learned model alone): {learned_target_rank}")

## Step 10: Ensemble - Combining History + Learned

The final prediction combines both components:

**Formula**: `final_score = history_score + model_weight × normalized_learned_score`

The normalization ensures both components are on comparable scales, and the learned weights determine their relative importance.

In [ ]:
print("=" * 80)
print("ENSEMBLE PREDICTION")
print("=" * 80)

# Get model weights
recency_decay = model.recency_decay.item()
freq_weight = model.freq_weight.item()
history_scale = model.history_scale.item()
model_weight = model.model_weight.item()

print(f"\nEnsemble Parameters:")
print(f"  History scale: {history_scale:.2f}x")
print(f"  Model weight: {model_weight:.2f}x")
print(f"  Ratio (history:learned): {history_scale/model_weight:.1f}:1")

# Compute final predictions (same as model forward)
with torch.no_grad():
    # Normalize learned logits
    learned_normalized = F.softmax(learned_logits, dim=1) * num_locations
    
    # Combine
    final_logits = history_scores + model_weight * learned_normalized
    
    # Get final predictions for first sample
    final_logits_sample = final_logits[0].cpu().numpy()

# Analyze final predictions
top_final_indices = np.argsort(final_logits_sample)[-10:][::-1]
top_final_scores = final_logits_sample[top_final_indices]

print(f"\nTop 10 Final Predictions:")
print(f"{'Rank':<6} {'Loc ID':<8} {'Final':<10} {'History':<10} {'Learned':<10} {'In Seq':<8} {'Notes'}")
print("-" * 80)

for i, loc_id in enumerate(top_final_indices, 1):
    final_sc = final_logits_sample[loc_id]
    hist_sc = history_scores_np[loc_id]
    learn_sc = learned_normalized[0, loc_id].cpu().item()
    in_seq = "✓" if loc_id in loc_seq.tolist() else "✗"
    target_marker = "← TARGET!" if loc_id == target.item() else ""
    
    print(f"{i:<6} {loc_id:<8} {final_sc:<10.2f} {hist_sc:<10.2f} {learn_sc:<10.2f} {in_seq:<8} {target_marker}")

# Target analysis
final_target_score = final_logits_sample[target.item()]
final_target_rank = np.sum(final_logits_sample > final_target_score) + 1

print(f"\n{'='*80}")
print(f"TARGET LOCATION ANALYSIS")
print(f"{'='*80}")
print(f"Target Location ID: {target.item()}")
print(f"  History score:  {history_scores_np[target.item()]:7.2f}")
print(f"  Learned score:  {learned_normalized[0, target.item()].cpu().item():7.2f}")
print(f"  Final score:    {final_target_score:7.2f}")
print(f"  Final rank:     {final_target_rank}")
print(f"  In history:     {is_in_history}")
print(f"\n  Prediction: {'✓ CORRECT' if final_target_rank == 1 else '✗ INCORRECT'}")

## Step 11: Evaluate Full Test Set

Now let's run the complete model on all test samples and compute aggregate metrics.

In [ ]:
print("=" * 80)
print("EVALUATING ON FULL TEST SET")
print("=" * 80)

# Initialize metric accumulators
metrics = {
    "correct@1": 0,
    "correct@3": 0,
    "correct@5": 0,
    "correct@10": 0,
    "rr": 0,
    "ndcg": 0,
    "f1": 0,
    "total": 0
}

# Lists for F1 score calculation
true_ls = []
top1_ls = []

# Evaluate on all batches
model.eval()
with torch.no_grad():
    for batch_idx, batch in enumerate(test_loader):
        # Move to device
        loc_seq = batch['loc_seq'].to(device)
        user_seq = batch['user_seq'].to(device)
        weekday_seq = batch['weekday_seq'].to(device)
        start_min_seq = batch['start_min_seq'].to(device)
        dur_seq = batch['dur_seq'].to(device)
        diff_seq = batch['diff_seq'].to(device)
        target = batch['target'].to(device)
        mask = batch['mask'].to(device)
        
        # Forward pass
        logits = model(loc_seq, user_seq, weekday_seq, start_min_seq, dur_seq, diff_seq, mask)
        
        # Calculate metrics
        result, batch_true, batch_top1 = calculate_correct_total_prediction(logits, target)
        
        metrics["correct@1"] += result[0]
        metrics["correct@3"] += result[1]
        metrics["correct@5"] += result[2]
        metrics["correct@10"] += result[3]
        metrics["rr"] += result[4]
        metrics["ndcg"] += result[5]
        metrics["total"] += result[6]
        
        # Collect for F1 score
        true_ls.extend(batch_true.tolist())
        if not batch_top1.shape:
            top1_ls.extend([batch_top1.tolist()])
        else:
            top1_ls.extend(batch_top1.tolist())
        
        if (batch_idx + 1) % 100 == 0:
            print(f"  Processed {batch_idx + 1}/{len(test_loader)} batches...")

# Calculate F1 score
f1 = f1_score(true_ls, top1_ls, average="weighted")
metrics["f1"] = f1

# Calculate final percentages
perf = get_performance_dict(metrics)

print(f"\n{'='*80}")
print(f"FINAL TEST SET PERFORMANCE")
print(f"{'='*80}")
print(f"\nTotal samples evaluated: {int(perf['total'])}")
print(f"\nAccuracy Metrics:")
print(f"  Acc@1:  {perf['acc@1']:6.2f}%  (exact match)")
print(f"  Acc@5:  {perf['acc@5']:6.2f}%  (in top 5)")
print(f"  Acc@10: {perf['acc@10']:6.2f}%  (in top 10)")
print(f"\nRanking Metrics:")
print(f"  MRR:    {perf['mrr']:6.2f}%  (mean reciprocal rank)")
print(f"  NDCG:   {perf['ndcg']:6.2f}%  (normalized DCG)")
print(f"\nClassification Metric:")
print(f"  F1:     {perf['f1']*100:6.2f}%  (weighted F1 score)")
print(f"\n{'='*80}")

## Step 12: Visualize Prediction Distribution

Let's visualize how the model's predictions are distributed across different accuracy levels.

In [ ]:
# Analyze prediction distribution
correct_at_k = {
    'Top-1': perf['correct@1'],
    'Top-3': perf['correct@3'] - perf['correct@1'],
    'Top-5': perf['correct@5'] - perf['correct@3'],
    'Top-10': perf['correct@10'] - perf['correct@5'],
    'Beyond Top-10': perf['total'] - perf['correct@10']
}

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of prediction ranks
categories = list(correct_at_k.keys())
values = list(correct_at_k.values())
colors = ['#2ecc71', '#3498db', '#f39c12', '#e74c3c', '#95a5a6']

ax1.bar(categories, values, color=colors, alpha=0.8, edgecolor='black')
ax1.set_ylabel('Number of Predictions', fontsize=12)
ax1.set_title('Distribution of Correct Predictions by Rank', fontsize=14, fontweight='bold')
ax1.set_ylim(0, max(values) * 1.1)
ax1.grid(axis='y', alpha=0.3)

for i, v in enumerate(values):
    ax1.text(i, v + max(values)*0.02, f'{int(v)}\n({v/perf["total"]*100:.1f}%)', 
             ha='center', va='bottom', fontweight='bold')

# Cumulative accuracy
cumulative_acc = [
    perf['acc@1'],
    perf['correct@3'] / perf['total'] * 100,
    perf['acc@5'],
    perf['acc@10'],
    100.0
]

ax2.plot(range(1, 6), cumulative_acc, marker='o', linewidth=2, markersize=10, color='#2ecc71')
ax2.fill_between(range(1, 6), cumulative_acc, alpha=0.3, color='#2ecc71')
ax2.set_xlabel('Top-K', fontsize=12)
ax2.set_ylabel('Cumulative Accuracy (%)', fontsize=12)
ax2.set_title('Cumulative Accuracy by Rank', fontsize=14, fontweight='bold')
ax2.set_xticks(range(1, 6))
ax2.set_xticklabels(['1', '3', '5', '10', 'All'])
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 105)

for i, (x, y) in enumerate(zip(range(1, 6), cumulative_acc)):
    ax2.annotate(f'{y:.1f}%', xy=(x, y), xytext=(0, 10), 
                textcoords='offset points', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('/content/expr_hrcl_next_pred_av5/notebooks/model_predictions_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("Visualization saved to: notebooks/model_predictions_analysis.png")

## Step 13: Analyze History Coverage

Since the model relies heavily on visit history, let's analyze what percentage of targets are actually in the history.

In [ ]:
# Analyze history coverage
history_coverage = []

with torch.no_grad():
    for batch in test_loader:
        loc_seq = batch['loc_seq']
        target = batch['target']
        
        for i in range(loc_seq.shape[0]):
            seq = loc_seq[i].tolist()
            tgt = target[i].item()
            
            # Check if target is in sequence
            is_in_hist = tgt in seq
            history_coverage.append(is_in_hist)

history_coverage_pct = sum(history_coverage) / len(history_coverage) * 100

print("=" * 80)
print("HISTORY COVERAGE ANALYSIS")
print("=" * 80)
print(f"\nTotal test samples: {len(history_coverage)}")
print(f"Targets in history: {sum(history_coverage)} ({history_coverage_pct:.2f}%)")
print(f"Targets NOT in history: {len(history_coverage) - sum(history_coverage)} ({100-history_coverage_pct:.2f}%)")
print(f"\nThis validates the model's design principle:")
print(f"  → {history_coverage_pct:.1f}% of next locations are in visit history")
print(f"  → History-based scoring is highly effective for this task")
print("=" * 80)

## Summary: How the Model Works

### 🔍 Complete Process Flow

1. **Input**: Trajectory sequence with location IDs, user info, and temporal features

2. **History Scoring** (Primary Component ~83% weight):
   - Compute recency scores: Recent visits get higher weights
   - Compute frequency scores: Often-visited places get higher weights
   - Combine and scale: `history_score = scale × (recency + freq_weight × frequency)`

3. **Learned Model** (Secondary Component ~17% weight):
   - Embed locations and users into dense vectors
   - Encode temporal features using circular representations
   - Apply self-attention to capture sequential patterns
   - Generate learned scores for all locations

4. **Ensemble**:
   - Normalize learned scores to comparable scale
   - Combine: `final = history_scores + model_weight × learned_scores`
   - History dominates (~11x vs ~0.22x for learned)

5. **Prediction**:
   - Select location with highest final score
   - Report top-K for evaluation

### 🎯 Key Performance Characteristics

The model achieves strong performance through its hybrid approach:
- Leverages the statistical insight that most next locations are revisits
- Uses learned patterns for novel locations and temporal dependencies
- Remains compact (under 500K parameters) while being highly effective

### 💡 Design Rationale

The model's architecture reflects a key insight from data analysis: **most next locations are revisits**. By heavily weighting history-based scoring while still learning sequential patterns, the model achieves high accuracy with a compact parameter budget.

The learned component handles:
- Novel locations (not in history)
- Temporal patterns (time-of-day, day-of-week effects)
- User-specific preferences
- Transition patterns between locations

This hybrid approach outperforms purely learned models while remaining interpretable and parameter-efficient.

---

## 🎓 Notebook Complete!

You've now seen every step of how the History-Centric model processes inputs and generates predictions. The notebook is fully self-contained and can be run independently to reproduce all results.